In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets evaluate

In [ ]:
!pip install nlpaug nltk


In [ ]:
import os
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import (
    XLNetTokenizerFast,
    XLNetForTokenClassification,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification,
    TrainerCallback,
)
from torch.utils.data import DataLoader
import evaluate
import wandb
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
wandb.login(key="****")
wandb.init(project="NER",
           name="xlnet-NER")


In [ ]:
# Load the CoNLL-2003 dataset
dataset = load_dataset("conll2003")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
# Initialize the XLNet tokenizer
tokenizer = XLNetTokenizerFast.from_pretrained("xlnet-base-cased")

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [ ]:
# Tokenization with Debugging for Alignment
def preprocess_data(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
        labels.append(label_ids)

        # Debug: Ensuring token-label alignment
        if i < 1:
            print("Tokens:", tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][i]))
            print("Labels:", label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Data Augmentation
from nlpaug.augmenter.word import SynonymAug

def augment_data(example):
    aug = SynonymAug(aug_src="wordnet", aug_max=3)  # Initialize the augmenter
    augmented_tokens = aug.augment(example["tokens"])  # Augment the tokens
    return {
        "tokens": augmented_tokens,  # Replace tokens with augmented tokens
        "ner_tags": example["ner_tags"],  # Keep the original NER tags
    }


In [ ]:
# Apply augmentation to training set
dataset["train"] = dataset["train"].map(augment_data)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

In [ ]:
# Tokenize the dataset
tokenized_datasets = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Tokens: ['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Tokens: ['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '▁CR', 'ICK', 'ET', '▁', '-', '▁', 'LE', 'ICE', 'S', 'TER', 'SH', 'IRE', '▁T', 'AKE', '▁', 'OVER', '▁AT', '▁', 'TOP', '▁', 'AF', 'TER', 

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Tokens: ['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '▁SO', 'C', 'CER', '▁', '-', '▁', 'JA', 'PAN', '▁', 'GET', '▁L', 'UC', 'KY', '▁', 'WIN', '▁', ',', '▁CH', 'INA', '▁IN', '▁', 'S

In [ ]:
# Initialize XLNet model for token classification
num_labels = len(dataset["train"].features["ner_tags"].feature.names)
model = XLNetForTokenClassification.from_pretrained("xlnet-base-cased", num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForTokenClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Advanced Training Arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NER/Result",
    run_name="xlnet-NER",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=15,
    weight_decay=0.001,
    save_total_limit=2,
    logging_dir="/content/drive/MyDrive/NER/Result",
    logging_steps=10,
    report_to="wandb",  # Report logs and metrics to W&B
    fp16=True,


In [ ]:
# Metrics Calculation
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_predictions = [
        [pred for pred, label in zip(pred_row, label_row) if label != -100]
        for pred_row, label_row in zip(predictions, labels)
    ]
    true_labels = [
        [label for pred, label in zip(pred_row, label_row) if label != -100]
        for pred_row, label_row in zip(predictions, labels)
    ]

    # Flatten lists
    true_predictions = [item for sublist in true_predictions for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]

    # Compute metrics
    precision = precision_metric.compute(predictions=true_predictions, references=true_labels, average="weighted")["precision"]
    recall = recall_metric.compute(predictions=true_predictions, references=true_labels, average="weighted")["recall"]
    f1 = f1_metric.compute(predictions=true_predictions, references=true_labels, average="weighted")["f1"]
    accuracy = accuracy_metric.compute(predictions=true_predictions, references=true_labels)["accuracy"]

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy,
    }


In [ ]:
# Custom Callback for Dynamic Learning Rate Adjustment
class DynamicLearningRateCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, logs=None, **kwargs):
        current_lr = args.learning_rate * (0.95 ** state.epoch)
        print(f"New Learning Rate: {current_lr}")
        control.should_save = True  # Save checkpoints after dynamic adjustment


In [ ]:
# Data Collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[DynamicLearningRateCallback()],
)



In [ ]:
# Train the Model
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.095400,0.106946,0.970429,0.969510,0.969369,0.969510
2,0.031100,0.127487,0.970653,0.970160,0.970071,0.970160
3,0.032500,0.137818,0.971836,0.970823,0.971111,0.970823
4,0.017600,0.161264,0.970026,0.966592,0.967822,0.966592
5,0.014600,0.150647,0.971060,0.970326,0.970514,0.970326
6,0.009700,0.166139,0.971186,0.970408,0.970569,0.970408
7,0.008400,0.185396,0.970221,0.968846,0.969268,0.968846
8,0.004000,0.177640,0.972568,0.971459,0.971858,0.971459
9,0.005600,0.194624,0.970426,0.969994,0.970105,0.969994
10,0.002500,0.196333,0.972168,0.971598,0.971704,0.971598


New Learning Rate: 4.75e-05
New Learning Rate: 4.5125e-05
New Learning Rate: 4.2868749999999995e-05
New Learning Rate: 4.07253125e-05
New Learning Rate: 3.868904687499999e-05
New Learning Rate: 3.675459453124999e-05
New Learning Rate: 3.4916864804687486e-05
New Learning Rate: 3.3171021564453116e-05
New Learning Rate: 3.151247048623045e-05
New Learning Rate: 2.9936846961918937e-05
New Learning Rate: 2.8440004613822984e-05
New Learning Rate: 2.7018004383131835e-05
New Learning Rate: 2.5667104163975243e-05
New Learning Rate: 2.4383748955776477e-05
New Learning Rate: 2.3164561507987653e-05


TrainOutput(global_step=3300, training_loss=0.014615683729379353, metrics={'train_runtime': 3209.7731, 'train_samples_per_second': 65.617, 'train_steps_per_second': 1.028, 'total_flos': 1.490535610602624e+16, 'train_loss': 0.014615683729379353, 'epoch': 15.0})

In [ ]:
# Evaluate the Model
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.22408147156238556, 'eval_precision': 0.9723961403179741, 'eval_recall': 0.9713349373599933, 'eval_f1': 0.9716891283084788, 'eval_accuracy': 0.9713349373599933, 'eval_runtime': 27.1182, 'eval_samples_per_second': 119.846, 'eval_steps_per_second': 1.881, 'epoch': 15.0}


In [ ]:
# Generate Classification Report
test_predictions, test_labels, _ = trainer.predict(tokenized_datasets["test"])
test_predictions = test_predictions.argmax(axis=-1)

true_labels = [
    [label for label in label_row if label != -100]
    for label_row in test_labels
]
predicted_labels = [
    [pred for pred, label in zip(pred_row, label_row) if label != -100]
    for pred_row, label_row in zip(test_predictions, test_labels)
]
print("Classification Report:")
from sklearn.metrics import classification_report
print(classification_report(
    [label for sublist in true_labels for label in sublist],
    [pred for sublist in predicted_labels for pred in sublist]
))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     53058
           1       0.90      0.94      0.92      2710
           2       0.97      0.99      0.98      2465
           3       0.82      0.88      0.85      3215
           4       0.65      0.84      0.73      1208
           5       0.85      0.86      0.85      2632
           6       0.73      0.76      0.74       358
           7       0.68      0.66      0.67      1162
           8       0.44      0.54      0.48       315

    accuracy                           0.96     67123
   macro avg       0.78      0.83      0.80     67123
weighted avg       0.96      0.96      0.96     67123



In [ ]:
model.save_pretrained("NER")
tokenizer.save_pretrained("NER_tf")

('NER_tf/tokenizer_config.json',
 'NER_tf/special_tokens_map.json',
 'NER_tf/spiece.model',
 'NER_tf/added_tokens.json',
 'NER_tf/tokenizer.json')

In [ ]:
import shutil

shutil.copytree('/content/NER', '/content/drive/MyDrive/NER/Model/NER')
shutil.copytree('/content/NER_tf', '/content/drive/MyDrive/NER/Model/NER_tf')

'/content/drive/MyDrive/NER/Model/NER_tf'

In [ ]:
from transformers import XLNetForTokenClassification, XLNetTokenizerFast

# Load the saved model and tokenizer
model = XLNetForTokenClassification.from_pretrained("/content/drive/MyDrive/NER/Model/NER")
tokenizer = XLNetTokenizerFast.from_pretrained("/content/drive/MyDrive/NER/Model/NER_tf")

# Define the input text
input_text = "John lives in New York."

# Tokenize the input text (explicitly define max_length and enable truncation)
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=128
)

# Perform inference
outputs = model(**inputs)

# Get the predicted label IDs
predictions = outputs.logits.argmax(-1)

# Map label IDs to label names (create this mapping using your dataset's label set)
label_map = {i: label for i, label in enumerate(dataset["train"].features["ner_tags"].feature.names)}


# Convert token IDs back to tokens and map predictions to labels
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze())
predicted_labels = [label_map[label] for label in predictions.squeeze().tolist()]

tokens = [token.replace("▁", "") for token in tokens]

# Filter out tokens with label "O"
filtered_results = [(token, label) for token, label in zip(tokens, predicted_labels) if label != "O"]

# Display filtered results
print("Named Entities:")
for token, label in filtered_results:
    print(f"{token}: {label}")

Named Entities:
John: B-PER
New: B-LOC
York: I-LOC
